In [1]:
import pandas as pd
import numpy as np
import pickle
import torch
import torch.nn as nn
from torch.nn import functional as F
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from torch.nn.utils.rnn import pad_sequence

In [9]:
# import training and validation data (encoded data)
train_data = torch.tensor(np.fromfile('train.bin', dtype=np.int16), dtype=torch.long)
val_data = torch.tensor(np.fromfile('val.bin', dtype=np.int16), dtype=torch.long)

# import tokens
with open('tokens.pickle', 'rb') as f:
    tokens = pickle.load(f)
vocab_size = len(tokens)

# encoder / decoder
# consider removing spaces depending on results
with open('encoder.pickle', 'rb') as f:
    encoder = pickle.load(f)
decoder = {v:k for k,v in encoder.items()}
decoder[0] = '0'
decoder[2] = '\n'

# update this to split at <EOS>
decode = lambda l: ' '.join([decoder[i] for i in l])  


In [3]:
# hyperparameters

batch_size = 32
block_size = 16
max_iters = 5000
eval_interval = 50
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 6
n_layer = 6
dropout = 0.2

# ---------------
torch.manual_seed(1337)
device

'cuda'

In [4]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

    
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

    
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

    
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [5]:
%%time

model = GPTLanguageModel()
m = model.to(device)

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

1.216384 M parameters
step 0: train loss 5.1695, val loss 5.1718
step 50: train loss 1.9759, val loss 1.9199
step 100: train loss 1.8841, val loss 1.8661
step 150: train loss 1.8614, val loss 1.8255
step 200: train loss 1.8296, val loss 1.7934
step 250: train loss 1.8068, val loss 1.7819
step 300: train loss 1.7847, val loss 1.7555
step 350: train loss 1.7715, val loss 1.7467
step 400: train loss 1.7508, val loss 1.7360
step 450: train loss 1.7557, val loss 1.7352
step 500: train loss 1.7460, val loss 1.7214
step 550: train loss 1.7262, val loss 1.7134
step 600: train loss 1.7206, val loss 1.6816
step 650: train loss 1.7001, val loss 1.6859
step 700: train loss 1.6862, val loss 1.6805
step 750: train loss 1.6888, val loss 1.6723
step 800: train loss 1.6876, val loss 1.6892
step 850: train loss 1.6701, val loss 1.6604
step 900: train loss 1.6686, val loss 1.6633
step 950: train loss 1.6611, val loss 1.6573
step 1000: train loss 1.6508, val loss 1.6432
step 1050: train loss 1.6542, val l

In [24]:
# generate from the model
context = torch.ones((1, 1), dtype=torch.long, device=device)
generated = decode(m.generate(context, max_new_tokens=100)[0].tolist())
print(generated)

  F C G Am F C 
 C Am7 F G 
 G C G C G C G Am Em F C 
 C G 
 C C5 Am F 
 Am C F G Am C 
 Am G C 
 C Fadd9 Am7 Fadd9 
 C Dm F C Dm F 
 C Am G F C Am F Dm C Dm 
 F G 
 G Am F 
 F Am G 
 Am C F G 
 C Em Am G F Em Dm G 
 C G F 
 G F C 
 C
